In [1]:
from skimage.io import imread
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu,gaussian
from skimage.transform import resize
from skimage import color
from numpy import asarray
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
import joblib

In [2]:
class ECG:
    def  getImage(self,image):
        """
        this functions gets user image
        return: user image
        """
        image=imread(image)
        return image

    def DividingLeads(self,image):
        """
        This Function Divides the Ecg image into 13 Leads including long lead. 
        Bipolar limb leads(Leads1,2,3). 
        Augmented unipolar limb leads(aVR,aVF,aVL). 
        Unipolar (+) chest leads(V1,V2,V3,V4,V5,V6)
        return : List containing all 13 leads divided
        """
        
        Lead_1 = image[300:600, 150:643]
        Lead_2 = image[300:600, 646:1135]
        Lead_3 = image[300:600, 1140:1625]
        Lead_4 = image[300:600, 1630:2125]
        Lead_5 = image[600:900, 150:643]
        Lead_6 = image[600:900, 646:1135]
        Lead_7 = image[600:900, 1140:1625]
        Lead_8 = image[600:900, 1630:2125]
        Lead_9 = image[900:1200, 150:643]
        Lead_10 = image[900:1200, 646:1135]
        Lead_11 = image[900:1200, 1140:1625]
        Lead_12 = image[900:1200, 1630:2125]
        Lead_13 = image[1250:1480, 150:2125]

        Leads=[Lead_1,Lead_2,Lead_3,Lead_4,Lead_5,Lead_6,Lead_7,Lead_8,Lead_9,Lead_10,Lead_11,Lead_12, Lead_13]
        
        return Leads
    
    def PreprocessingLeads(self,Leads):
        """
        This Function Performs preprocessing to on the extracted leads.
        """
        
        bin_glob=[]


        #looping through image list containg all leads from 1-12
        for x,y in enumerate(Leads[:len(Leads)-1]):

            #converting to gray scale
            grayscale = color.rgb2gray(y)

            #smoothing image
            blurred_image = gaussian(grayscale, sigma=0.7)
            #thresholding to distinguish foreground and background
            #using otsu thresholding for getting threshold value
            global_thresh = threshold_otsu(blurred_image)

            #creating binary image based on threshold
            binary_global = blurred_image < global_thresh

            #resize image
            binary_global = resize(binary_global, (300, 450))
            bin_glob.append(binary_global)
            
        return bin_glob
    
    def SignalExtraction_Scaling(self,bin_glob):
        """
        This Function Performs Signal Extraction using various steps,techniques: convert to grayscale, 
        apply gaussian filter, thresholding, perform contouring to extract signal image and then save the image as 1D signal
        """
        
        resized_leads=[]

        for i in bin_glob:

            # Find the non-zero coordinates

            combo = np.transpose(np.nonzero(i))
            resized_combo = np.resize(combo, (255, 2))
            resized_combo = sorted(resized_combo, key=lambda x: x[1], reverse=True)

            # Print the resized combo array
            resized_leads.append(resized_combo)
        
        return resized_leads
    
    def CombineConvert1Dsignal(self, resized_leads):
        scaled_leads=[]


        for i in range(12):
            fit_transform_data = scaler.fit_transform(resized_leads[i])
            Normalized_Scaled=pd.DataFrame(fit_transform_data, columns = ['X','Y'])
            scaled_leads=scaled_leads + Normalized_Scaled['X'].tolist()
            
        return scaled_leads
    
    def DimensionalReduction(self,scaled_leads):

        pca_loaded_model = joblib.load('PCA_ECG (1).pkl')
        result = pca_loaded_model.transform(pd.DataFrame(scaled_leads).T)
        result = pd.DataFrame(result)
        return result


    def ModelLoad_predict(self,result):
        
        loaded_model = joblib.load('D:\\Cardiovascular-Detection-using-ECG-images-main\\model_test_cv_python_3.pkl') #
        result = loaded_model.predict(result)
        
        return result
    


In [3]:
#%%timeit
import warnings
import os
warnings.filterwarnings("ignore")
count_HB=0
count_MI=0
count_PMI=0
count_NO=0
unclassified=0
count=[]
total=0

#signals_pc_python=[]


# Import time module
import time
 
# record start time
start = time.time()

for i in os.listdir("D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET"):
    for j in os.listdir(f'D:\\Cardiovascular-Detection-using-ECG-images-main\\ECG_IMAGES_DATASET/{i}'):

        #from Ecg import  ECG
        #intialize ecg object
        ecg= ECG()
        
        uploaded_file = f"D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/{i}/{j}"
        print(uploaded_file)
        
        image = ecg.getImage(uploaded_file)
        dividing_leads = ecg.DividingLeads(image)
        ecg_preprocessed_leads = ecg.PreprocessingLeads(dividing_leads)
        ecg_signal_extraction=ecg.SignalExtraction_Scaling(ecg_preprocessed_leads)
        ecg_1dsignal=ecg.CombineConvert1Dsignal(ecg_signal_extraction)
        ecg_final=ecg.DimensionalReduction(ecg_1dsignal)
        ecg_model=ecg.ModelLoad_predict(ecg_final)
        
        #signals_pc_python.append(ecg_final)

       

        if ecg_model == [2] and i=='Abnormal':  #0   
            count_HB=count_HB+1
            print("ECG corresponds to Abnormal ",count_HB)
           

        elif ecg_model == [0] and i=='MI':  #1   
            count_MI=count_MI+1
            print("ECG corresponds to Myocardial Infarction(MI) ",count_MI)
            

        elif ecg_model == [1] and i=='Normal':  #2    
            count_NO=count_NO+1
            print("ECG corresponds to Normal",count_NO)
           

        elif  ecg_model == [3] and i=='PMI': #   
            count_PMI=count_PMI+1
            print("ECG corresponds to History of Myocardial Infarction(PMI) ",count_PMI)
            
        else:
            unclassified=unclassified+1
            print("unclassified ", unclassified)
            
                    
end = time.time()


D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(1).jpg
ECG corresponds to Abnormal  1
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(10).jpg
ECG corresponds to Abnormal  2
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(100).jpg
ECG corresponds to Abnormal  3
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(101).jpg
ECG corresponds to Abnormal  4
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(102).jpg
ECG corresponds to Abnormal  5
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(103).jpg
ECG corresponds to Abnormal  6
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(104).jpg
ECG corresponds to Abnormal  7
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(105).jpg
unclassified  1
D:/Cardiovascular-Detection-using-ECG-images-main/

ECG corresponds to Abnormal  68
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(161).jpg
ECG corresponds to Abnormal  69
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(162).jpg
ECG corresponds to Abnormal  70
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(163).jpg
ECG corresponds to Abnormal  71
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(164).jpg
ECG corresponds to Abnormal  72
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(165).jpg
ECG corresponds to Abnormal  73
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(166).jpg
ECG corresponds to Abnormal  74
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(167).jpg
ECG corresponds to Abnormal  75
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(168).jpg
ECG corresponds to Abnor

unclassified  10
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(223).jpg
ECG corresponds to Abnormal  129
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(224).jpg
ECG corresponds to Abnormal  130
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(225).jpg
ECG corresponds to Abnormal  131
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(226).jpg
ECG corresponds to Abnormal  132
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(227).jpg
ECG corresponds to Abnormal  133
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(228).jpg
ECG corresponds to Abnormal  134
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(229).jpg
ECG corresponds to Abnormal  135
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(23).jpg
ECG corresponds to Abnormal  136


ECG corresponds to Abnormal  189
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(76).jpg
ECG corresponds to Abnormal  190
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(77).jpg
ECG corresponds to Abnormal  191
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(78).jpg
ECG corresponds to Abnormal  192
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(79).jpg
ECG corresponds to Abnormal  193
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(8).jpg
ECG corresponds to Abnormal  194
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(80).jpg
ECG corresponds to Abnormal  195
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(81).jpg
ECG corresponds to Abnormal  196
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/HB(82).jpg
unclassified  19
D:/Cardi

ECG corresponds to Myocardial Infarction(MI)  39
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/MI/MI(134).jpg
ECG corresponds to Myocardial Infarction(MI)  40
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/MI/MI(135).jpg
ECG corresponds to Myocardial Infarction(MI)  41
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/MI/MI(136).jpg
ECG corresponds to Myocardial Infarction(MI)  42
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/MI/MI(137).jpg
ECG corresponds to Myocardial Infarction(MI)  43
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/MI/MI(138).jpg
ECG corresponds to Myocardial Infarction(MI)  44
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/MI/MI(139).jpg
ECG corresponds to Myocardial Infarction(MI)  45
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/MI/MI(14).jpg
ECG corresponds to Myocardial Infarction(MI)  46
D:/Cardiovascular-Det

...

ECG corresponds to Normal 182
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Normal/Normal(273).jpg
ECG corresponds to Normal 183
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Normal/Normal(274).jpg
ECG corresponds to Normal 184
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Normal/Normal(275).jpg
ECG corresponds to Normal 185
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Normal/Normal(276).jpg
ECG corresponds to Normal 186
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Normal/Normal(277).jpg
ECG corresponds to Normal 187
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Normal/Normal(278).jpg
ECG corresponds to Normal 188
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Normal/Normal(279).jpg
ECG corresponds to Normal 189
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Normal/Normal(28).jpg
ECG corresponds to Normal

unclassified  37
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Normal/Normal(8).jpg
ECG corresponds to Normal 246
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Normal/Normal(80).jpg
ECG corresponds to Normal 247
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Normal/Normal(81).jpg
ECG corresponds to Normal 248
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Normal/Normal(82).jpg
ECG corresponds to Normal 249
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Normal/Normal(83).jpg
ECG corresponds to Normal 250
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Normal/Normal(84).jpg
ECG corresponds to Normal 251
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Normal/Normal(85).jpg
ECG corresponds to Normal 252
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Normal/Normal(86).jpg
ECG corresponds to Normal 253
D:/Cardiovascula

ECG corresponds to History of Myocardial Infarction(PMI)  36
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI/PMI(136).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  37
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI/PMI(137).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  38
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI/PMI(138).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  39
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI/PMI(139).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  40
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI/PMI(14).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  41
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI/PMI(140).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  42
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAG

ECG corresponds to History of Myocardial Infarction(PMI)  85
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI/PMI(35).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  86
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI/PMI(36).jpg
unclassified  55
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI/PMI(37).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  87
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI/PMI(38).jpg
unclassified  56
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI/PMI(39).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  88
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI/PMI(4).jpg
unclassified  57
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI/PMI(40).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  89
D:/Cardiovascular-Detection-using-ECG-images-main/

ECG corresponds to History of Myocardial Infarction(PMI)  132
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI/PMI(9).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  133
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI/PMI(90).jpg
unclassified  68
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI/PMI(91).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  134
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI/PMI(92).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  135
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI/PMI(93).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  136
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI/PMI(94).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  137
D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI/PMI(95).jpg
ECG corresponds t

In [4]:
list=[count_NO/len(os.listdir('D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Normal/')),
                count_HB/len(os.listdir('D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal/')),
                    count_PMI/len(os.listdir('D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI/')),
                        count_MI/len(os.listdir('D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/MI/')),
           unclassified]
labels=['Normal','Abnormal','PMI','MI','Unclassified']
l=pd.DataFrame(list,labels)
l.columns=['Values']
l

,Values
Normal,0.936620
Abnormal,0.914163
PMI,0.825581
MI,1.000000
Unclassified,68.000000


In [5]:
print("The time of execution of above program is :", (end-start) * 10**3/60000 * 60, "seconds")

The time of execution of above program is : 193.57022356987 seconds


In [6]:
print("Overall Accuracy :- ", (count_NO/284+count_HB/233+count_PMI/172+count_MI/239)/4)

Overall Accuracy :-  0.9190910509468629
